<a href="https://colab.research.google.com/github/deekshant007/deekshant007/blob/main/file_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
!pip install ray
!pip install modin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [137]:
import os
import time

In [138]:
import modin.pandas as pd
import ray
ray.shutdown()
ray.init()
start = time.time()
df = pd.read_csv('/DP_LIVE_15082022195839888.csv')
end = time.time()
print("Read csv with modin and ray: ",(end-start),"sec")

Read csv with modin and ray:  0.8890230655670166 sec


In [139]:
from dask import dataframe as dd
start = time.time()
dask_df = dd.read_csv('/DP_LIVE_15082022195839888.csv')
end = time.time()
print("Read csv with dask: ",(end-start),"sec")

Read csv with dask:  0.0358426570892334 sec


In [140]:
from dask import dataframe as dd
df = dd.read_csv('/DP_LIVE_15082022195839888.csv',delimiter=',')

In [141]:
df.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 8 entries, LOCATION to Flag Codes
dtypes: object(5), float64(2), int64(1)

In [142]:
len(df.columns)

8

In [143]:
len(df.index)

3963

In [144]:
df.columns=df.columns.str.replace('[#,@,&]','')

In [145]:
df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,EXCH,TOT,NATUSD,A,1950,0.892857,NaN
1,AUS,EXCH,TOT,NATUSD,A,1951,0.892857,NaN
2,AUS,EXCH,TOT,NATUSD,A,1952,0.892857,NaN
3,AUS,EXCH,TOT,NATUSD,A,1953,0.892857,NaN
4,AUS,EXCH,TOT,NATUSD,A,1954,0.892857,NaN


In [146]:
df.columns = df.columns.str.replace(' ', '')

In [147]:
data=df.columns
data

Index(['LOCATION', 'INDICATOR', 'SUBJECT', 'MEASURE', 'FREQUENCY', 'TIME',
       'Value', 'FlagCodes'],
      dtype='object')

In [148]:
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

In [149]:
%%writefile utility.py

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

Overwriting utility.py


In [159]:
%%writefile store.yaml
file_type: csv
dataset_name: DP_LIVE_15082022195839888
file_name: DP_LIVE_15082022195839888
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - LOCATION
      - INDICATOR
      - SUBJECT
      - MEASURE
      - FREQUENCY
      - TIME
      - Value
      - Flag Codes

Overwriting store.yaml


In [160]:
def read_config_file(alpha):
    with open(alpha, 'r') as f:
        try:
            return yaml.load(f, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

In [190]:
def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    #df.columns = list(map(lambda x: replacel(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0   

In [176]:
config_data = read_config_file("store.yaml")

In [177]:
config_data

{'file_type': 'csv',
 'dataset_name': 'DP_LIVE_15082022195839888',
 'file_name': 'DP_LIVE_15082022195839888',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['LOCATION - INDICATOR - SUBJECT - MEASURE - FREQUENCY - TIME - Value - Flag Codes']}

In [178]:
from dask import dataframe as dd
df_sample = dd.read_csv('/content/DP_LIVE_15082022195839888.csv',delimiter=',')
df_sample.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,EXCH,TOT,NATUSD,A,1950,0.892857,NaN
1,AUS,EXCH,TOT,NATUSD,A,1951,0.892857,NaN
2,AUS,EXCH,TOT,NATUSD,A,1952,0.892857,NaN
3,AUS,EXCH,TOT,NATUSD,A,1953,0.892857,NaN
4,AUS,EXCH,TOT,NATUSD,A,1954,0.892857,NaN


In [179]:
file_type = config_data['file_type']
source_file = "/content/" + config_data['file_name'] + f'.{file_type}'

In [180]:
import pandas as pd
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,EXCH,TOT,NATUSD,A,1950,0.892857,NaN
1,AUS,EXCH,TOT,NATUSD,A,1951,0.892857,NaN
2,AUS,EXCH,TOT,NATUSD,A,1952,0.892857,NaN
3,AUS,EXCH,TOT,NATUSD,A,1953,0.892857,NaN
4,AUS,EXCH,TOT,NATUSD,A,1954,0.892857,NaN


In [192]:
col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['subject', 'measure', 'location', 'flag_codes', 'time', 'value', 'indicator', 'frequency']
Following YAML columns are not in the file uploaded ['location - indicator - subject - measure - frequency - time - value - flag codes']


0

In [182]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['location', 'indicator', 'subject', 'measure', 'frequency', 'time',
       'value', 'flag_codes'],
      dtype='object')
columns of YAML are: ['LOCATION - INDICATOR - SUBJECT - MEASURE - FREQUENCY - TIME - Value - Flag Codes']


In [191]:
if col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation failed
Following File columns are not in the YAML file ['subject', 'measure', 'location', 'flag_codes', 'time', 'value', 'indicator', 'frequency']
Following YAML columns are not in the file uploaded ['location - indicator - subject - measure - frequency - time - value - flag codes']
validation failed


In [198]:
import datetime
import csv
import gzip

from dask import dataframe as dd
df = dd.read_csv('/content/DP_LIVE_15082022195839888.csv',delimiter=',')

# Write csv in gz format in pipe separated text file (|)
df.to_csv('DP_LIVE_15082022195839888.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['/content/DP_LIVE_15082022195839888.csv.gz/0.part']

In [199]:
import os
entries = os.listdir('/content/DP_LIVE_15082022195839888.csv.gz')
for entry in entries:
    print(entry)

0.part


In [200]:
os.path.getsize('/content/DP_LIVE_15082022195839888.csv.gz')

4096